# Data Cleaning For Our Main Dataset

In [1]:
import pandas as pd

### Import the relevant .csv files.

In [36]:
main_dataset = pd.read_csv("data/athlete_events.csv")
noc_dataset = pd.read_csv("data/noc_regions.csv")

Exploratory data analysis for main dataset.

In [37]:
main_dataset.shape

(271116, 15)

In [38]:
main_dataset.dtypes

ID          int64
Name       object
Sex        object
Age       float64
Height    float64
Weight    float64
Team       object
NOC        object
Games      object
Year        int64
Season     object
City       object
Sport      object
Event      object
Medal      object
dtype: object

In [39]:
main_dataset.describe()

,ID,Age,Height,Weight,Year
count,271116.000000,261642.000000,210945.000000,208241.000000,271116.000000
mean,68248.954396,25.556898,175.338970,70.702393,1978.378480
std,39022.286345,6.393561,10.518462,14.348020,29.877632
min,1.000000,10.000000,127.000000,25.000000,1896.000000
25%,34643.000000,21.000000,168.000000,60.000000,1960.000000
50%,68205.000000,24.000000,175.000000,70.000000,1988.000000
75%,102097.250000,28.000000,183.000000,79.000000,2002.000000
max,135571.000000,97.000000,226.000000,214.000000,2016.000000


### Clean the NOC regions dataset due to missing values in "region" (replace with values under "notes" instead).

In [40]:
null_rows = noc_dataset[noc_dataset["region"].isnull()]
print(null_rows, "\n\nMissing values filled\n")

for index in null_rows.index:
    noc_dataset["region"][index] = noc_dataset["notes"][index]
    print(noc_dataset.loc[index])

     NOC region                 notes
168  ROT    NaN  Refugee Olympic Team
208  TUV    NaN                Tuvalu
213  UNK    NaN               Unknown 

Missing values filled

NOC                        ROT
region    Refugee Olympic Team
notes     Refugee Olympic Team
Name: 168, dtype: object
NOC          TUV
region    Tuvalu
notes     Tuvalu
Name: 208, dtype: object
NOC           UNK
region    Unknown
notes     Unknown
Name: 213, dtype: object


### Add a country name column (use NOC regions dataset; code to name conversion), then rename the columns for readability.

In [41]:
main_dataset = main_dataset.merge(noc_dataset, left_on = "NOC", right_on = "NOC")
main_dataset = main_dataset.rename(columns = {"NOC": "CountryCode", "region": "CountryName"})
main_dataset.head()

,ID,Name,Sex,Age,Height,Weight,Team,CountryCode,Games,Year,Season,City,Sport,Event,Medal,CountryName,notes
0,1,A Dijiang,M,24.0,180.0,80.0,China,CHN,1992 Summer,1992,Summer,Barcelona,Basketball,Basketball Men's Basketball,NaN,China,NaN
1,2,A Lamusi,M,23.0,170.0,60.0,China,CHN,2012 Summer,2012,Summer,London,Judo,Judo Men's Extra-Lightweight,NaN,China,NaN
2,602,Abudoureheman,M,22.0,182.0,75.0,China,CHN,2000 Summer,2000,Summer,Sydney,Boxing,Boxing Men's Middleweight,NaN,China,NaN
3,1463,Ai Linuer,M,25.0,160.0,62.0,China,CHN,2004 Summer,2004,Summer,Athina,Wrestling,"Wrestling Men's Lightweight, Greco-Roman",NaN,China,NaN
4,1464,Ai Yanhan,F,14.0,168.0,54.0,China,CHN,2016 Summer,2016,Summer,Rio de Janeiro,Swimming,Swimming Women's 200 metres Freestyle,NaN,China,NaN


### Weed out the only columns we need.

In [42]:
main_dataset = main_dataset[["CountryCode", "CountryName", "Year", "Season", "Medal"]]
main_dataset.head()

,CountryCode,CountryName,Year,Season,Medal
0,CHN,China,1992,Summer,NaN
1,CHN,China,2012,Summer,NaN
2,CHN,China,2000,Summer,NaN
3,CHN,China,2004,Summer,NaN
4,CHN,China,2016,Summer,NaN


### Check if there are any null values in the columns.

In [43]:
main_dataset.isnull().any()

CountryCode    False
CountryName    False
Year           False
Season         False
Medal           True
dtype: bool

##### Medal column has null values. We will be replacing it with the value 0.

In [44]:
main_dataset["Medal"].fillna(value = 0, inplace = True)
main_dataset.head()

,CountryCode,CountryName,Year,Season,Medal
0,CHN,China,1992,Summer,0
1,CHN,China,2012,Summer,0
2,CHN,China,2000,Summer,0
3,CHN,China,2004,Summer,0
4,CHN,China,2016,Summer,0


### Sort our dataset by year (numerical order), country (alphabetical order), and season (summer first, then winter), and reset indexing to 0.
Also there are some faulty country code values in the original dataset. This is not unexpected seeing as the csv file is over 270,000 rows long...

In [45]:
#change faulty country code values.
faulty_codes_dict = {"MAS": "MAL", "NBO": "MAL",
                    "TCH": "CZE",
                    "YUG": "SRB", "SCG": "SRB",
                    "TTO": "TRI", "WIF": "TRI",
                    "SAA": "GER", "GDR": "GER",
                    "URS": "RUS", "EUN": "RUS",
                    "VNM": "VIE",
                    "RHO": "ZIM",
                    "UAR": "SYR",
                    "YAR": "YEM", "YMD": "YEM",
                    "SSD": "SUD"}

main_dataset = main_dataset.replace(to_replace = faulty_codes_dict)

#sort by year and country
main_dataset = main_dataset.sort_values(by = ["Year", "CountryCode", "Season"])
main_dataset = main_dataset.reset_index(drop = True)
main_dataset.head()

,CountryCode,CountryName,Year,Season,Medal
0,AUS,Australia,1896,Summer,0
1,AUS,Australia,1896,Summer,Bronze
2,AUS,Australia,1896,Summer,Gold
3,AUS,Australia,1896,Summer,Gold
4,AUS,Australia,1896,Summer,0


In [46]:
main_dataset

,CountryCode,CountryName,Year,Season,Medal
0,AUS,Australia,1896,Summer,0
1,AUS,Australia,1896,Summer,Bronze
2,AUS,Australia,1896,Summer,Gold
3,AUS,Australia,1896,Summer,Gold
4,AUS,Australia,1896,Summer,0
...,...,...,...,...,...
266861,ZIM,Zimbabwe,2016,Summer,0
266862,ZIM,Zimbabwe,2016,Summer,0
266863,ZIM,Zimbabwe,2016,Summer,0
266864,ZIM,Zimbabwe,2016,Summer,0


Done cleaning main dataset.

# Create our Ultimate Dataset: Summer

### Brainstorming
- Sort primarily by year, then secondarily by country. 
- A reminder that later than 1992, Summer and Winter olympics no longer play the same year. Instead, they alternate 2 years.

### Columns we need for Summer Dataset
1. Country Code
- Year
- Season of Olympics
- Total Participants
- Gold Medals
- Silver Medals
- Bronze Medals
- Total Medals
### Columns for Predictors; data only from 1960s and above

9. GDP Per Capita
- Population Size
- Host City
- Medal Count from Same Season's Games (Winter if Summer_Dataset, and Summer if Winter_Dataset)
- Medal Count from Previous Season's Games

### Importing relevant .csv files from our data cleaning notebook for external excel files (GDP Per Capita, Host City, Population Size)

In [47]:
summer_gdp_dataset = pd.read_csv("predictordata/summer_gdp_dataset.csv")
summer_pop_dataset = pd.read_csv("predictordata/summer_pop_dataset.csv")
summer_host_dataset = pd.read_csv("predictordata/summer_host_dataset.csv")

### Create the 3 datasets.

In [66]:
olympics_cols = ["CountryCode", "Year", "OlympicsSeason", "TotalParticipants", "GoldMedals", "SilverMedals", "BronzeMedals", "TotalMedals", "Win%", "GDPPerCapita", "PopulationSize", "HostCity"]

summer_dataset = pd.DataFrame(columns = olympics_cols)
winter_dataset = pd.DataFrame(columns = olympics_cols)

summer_dataset.columns

Index(['CountryCode', 'Year', 'OlympicsSeason', 'TotalParticipants',
       'GoldMedals', 'SilverMedals', 'BronzeMedals', 'TotalMedals', 'Win%',
       'GDPPerCapita', 'PopulationSize', 'HostCity'],
      dtype='object')

### Get unique values for country and year and store them in arrays.

In [70]:
countries_list = main_dataset.CountryCode.unique()
countries_list.sort()
years_list = main_dataset.Year.unique()

print(countries_list, "\n")
print(years_list)

['AFG' 'AHO' 'ALB' 'ALG' 'AND' 'ANG' 'ANT' 'ANZ' 'ARG' 'ARM' 'ARU' 'ASA'
 'AUS' 'AUT' 'AZE' 'BAN' 'BAR' 'BDI' 'BEL' 'BEN' 'BHU' 'BIH' 'BIZ' 'BLR'
 'BOH' 'BOL' 'BOT' 'BRA' 'BRN' 'BRU' 'BUL' 'BUR' 'CAF' 'CAM' 'CAN' 'CAY'
 'CGO' 'CHA' 'CHI' 'CHN' 'CIV' 'CMR' 'COD' 'COK' 'COL' 'COM' 'CPV' 'CRC'
 'CRO' 'CRT' 'CUB' 'CYP' 'CZE' 'DEN' 'DJI' 'DMA' 'DOM' 'ECU' 'EGY' 'ERI'
 'ESA' 'ESP' 'EST' 'ETH' 'FIJ' 'FIN' 'FRA' 'FSM' 'GAB' 'GAM' 'GBR' 'GBS'
 'GEO' 'GEQ' 'GER' 'GHA' 'GRE' 'GRN' 'GUA' 'GUI' 'GUM' 'GUY' 'HAI' 'HKG'
 'HON' 'HUN' 'INA' 'IND' 'IOA' 'IRI' 'IRL' 'IRQ' 'ISL' 'ISR' 'ISV' 'ITA'
 'IVB' 'JAM' 'JOR' 'JPN' 'KAZ' 'KEN' 'KGZ' 'KIR' 'KOR' 'KOS' 'KSA' 'KUW'
 'LAO' 'LAT' 'LBA' 'LBR' 'LCA' 'LES' 'LIB' 'LIE' 'LTU' 'LUX' 'MAD' 'MAL'
 'MAR' 'MAW' 'MDA' 'MDV' 'MEX' 'MGL' 'MHL' 'MKD' 'MLI' 'MLT' 'MNE' 'MON'
 'MOZ' 'MRI' 'MTN' 'MYA' 'NAM' 'NCA' 'NED' 'NEP' 'NFL' 'NGR' 'NIG' 'NOR'
 'NRU' 'NZL' 'OMA' 'PAK' 'PAN' 'PAR' 'PER' 'PHI' 'PLE' 'PLW' 'PNG' 'POL'
 'POR' 'PRK' 'PUR' 'QAT' 'ROT' 'ROU' 'RSA' 'RUS' 'R

### Clean the main dataset for the summer dataset.

In [71]:
summer_main_dataset = main_dataset
summer_main_dataset = summer_main_dataset[summer_main_dataset.Season == "Summer"]
summer_main_dataset = summer_main_dataset.reset_index(drop = True)
summer_years_list = [year for year in years_list if year%4 == 0 or year == 1906]

print(summer_years_list)
summer_main_dataset.head()

[1896, 1900, 1904, 1906, 1908, 1912, 1920, 1924, 1928, 1932, 1936, 1948, 1952, 1956, 1960, 1964, 1968, 1972, 1976, 1980, 1984, 1988, 1992, 1996, 2000, 2004, 2008, 2012, 2016]


,CountryCode,CountryName,Year,Season,Medal
0,AUS,Australia,1896,Summer,0
1,AUS,Australia,1896,Summer,Bronze
2,AUS,Australia,1896,Summer,Gold
3,AUS,Australia,1896,Summer,Gold
4,AUS,Australia,1896,Summer,0


# Making the summer dataset.

In [72]:
i = 0
j = 0

for year in summer_years_list:
    for country in countries_list:
        total_participants = 0
        gold = 0
        silver = 0
        bronze = 0

        try: 
            while summer_main_dataset["CountryCode"][i] == country:
                if summer_main_dataset["Medal"][i] == "Gold":
                    gold += 1
                elif summer_main_dataset["Medal"][i] == "Silver":
                    silver += 1
                elif summer_main_dataset["Medal"][i] == "Bronze":
                    bronze += 1
                total_participants += 1
                i += 1
        except KeyError:
            pass
            
        if total_participants == 0:
            continue
        
        summer_dict = {"CountryCode": country, 
                       "Year": year, 
                       "OlympicsSeason": (year-1896)//4 + 1, 
                       "TotalParticipants": total_participants, 
                       "GoldMedals": gold, 
                       "SilverMedals": silver,
                       "BronzeMedals": bronze, 
                       "TotalMedals": gold + silver + bronze,
                       "Win%": round((gold+silver+bronze)/total_participants*100, 2)}
        
        #predictor data available
        if year >= 1960:
            if country in ["AHO", "TPE", "COK", "IOA", "ERI", "KOS", "ROT"]: #countries with no predictor data
                continue
                
            if summer_host_dataset["CountryCode"][j] == country and summer_host_dataset["Year"][j] == year:
                is_host_city = 1
                j += 1
            else:
                is_host_city = 0
            
            summer_dict.update({"GDPPerCapita": summer_gdp_dataset.loc[summer_gdp_dataset["CountryCode"] == country, str(year)].iloc[0],
                                "PopulationSize": summer_pop_dataset.loc[summer_pop_dataset["CountryCode"] == country, str(year)].iloc[0],
                                "HostCity": is_host_city})
        
        summer_dataset = summer_dataset.append(summer_dict, ignore_index = True)

summer_dataset.head()

/tmp/ipykernel_6891/864083591.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summer_dataset = summer_dataset.append(summer_dict, ignore_index = True)
/tmp/ipykernel_6891/864083591.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summer_dataset = summer_dataset.append(summer_dict, ignore_index = True)
/tmp/ipykernel_6891/864083591.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summer_dataset = summer_dataset.append(summer_dict, ignore_index = True)
/tmp/ipykernel_6891/864083591.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summer_dataset = summer_dataset.append(summer_dict, ignore_index = True)
/tmp/ipykernel_6891/

/tmp/ipykernel_6891/864083591.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summer_dataset = summer_dataset.append(summer_dict, ignore_index = True)
/tmp/ipykernel_6891/864083591.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summer_dataset = summer_dataset.append(summer_dict, ignore_index = True)
/tmp/ipykernel_6891/864083591.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summer_dataset = summer_dataset.append(summer_dict, ignore_index = True)
/tmp/ipykernel_6891/864083591.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summer_dataset = summer_dataset.append(summer_dict, ignore_index = True)
/tmp/ipykernel_6891/

/tmp/ipykernel_6891/864083591.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summer_dataset = summer_dataset.append(summer_dict, ignore_index = True)
/tmp/ipykernel_6891/864083591.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summer_dataset = summer_dataset.append(summer_dict, ignore_index = True)
/tmp/ipykernel_6891/864083591.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summer_dataset = summer_dataset.append(summer_dict, ignore_index = True)
/tmp/ipykernel_6891/864083591.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summer_dataset = summer_dataset.append(summer_dict, ignore_index = True)
/tmp/ipykernel_6891/

/tmp/ipykernel_6891/864083591.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summer_dataset = summer_dataset.append(summer_dict, ignore_index = True)
/tmp/ipykernel_6891/864083591.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summer_dataset = summer_dataset.append(summer_dict, ignore_index = True)
/tmp/ipykernel_6891/864083591.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summer_dataset = summer_dataset.append(summer_dict, ignore_index = True)
/tmp/ipykernel_6891/864083591.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summer_dataset = summer_dataset.append(summer_dict, ignore_index = True)
/tmp/ipykernel_6891/

/tmp/ipykernel_6891/864083591.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summer_dataset = summer_dataset.append(summer_dict, ignore_index = True)
/tmp/ipykernel_6891/864083591.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summer_dataset = summer_dataset.append(summer_dict, ignore_index = True)
/tmp/ipykernel_6891/864083591.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summer_dataset = summer_dataset.append(summer_dict, ignore_index = True)
/tmp/ipykernel_6891/864083591.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summer_dataset = summer_dataset.append(summer_dict, ignore_index = True)
/tmp/ipykernel_6891/

...

/tmp/ipykernel_6891/864083591.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summer_dataset = summer_dataset.append(summer_dict, ignore_index = True)
/tmp/ipykernel_6891/864083591.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summer_dataset = summer_dataset.append(summer_dict, ignore_index = True)
/tmp/ipykernel_6891/864083591.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summer_dataset = summer_dataset.append(summer_dict, ignore_index = True)
/tmp/ipykernel_6891/864083591.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summer_dataset = summer_dataset.append(summer_dict, ignore_index = True)
/tmp/ipykernel_6891/

/tmp/ipykernel_6891/864083591.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summer_dataset = summer_dataset.append(summer_dict, ignore_index = True)
/tmp/ipykernel_6891/864083591.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summer_dataset = summer_dataset.append(summer_dict, ignore_index = True)
/tmp/ipykernel_6891/864083591.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summer_dataset = summer_dataset.append(summer_dict, ignore_index = True)
/tmp/ipykernel_6891/864083591.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summer_dataset = summer_dataset.append(summer_dict, ignore_index = True)
/tmp/ipykernel_6891/

/tmp/ipykernel_6891/864083591.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summer_dataset = summer_dataset.append(summer_dict, ignore_index = True)
/tmp/ipykernel_6891/864083591.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summer_dataset = summer_dataset.append(summer_dict, ignore_index = True)
/tmp/ipykernel_6891/864083591.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summer_dataset = summer_dataset.append(summer_dict, ignore_index = True)
/tmp/ipykernel_6891/864083591.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summer_dataset = summer_dataset.append(summer_dict, ignore_index = True)
/tmp/ipykernel_6891/

/tmp/ipykernel_6891/864083591.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summer_dataset = summer_dataset.append(summer_dict, ignore_index = True)
/tmp/ipykernel_6891/864083591.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summer_dataset = summer_dataset.append(summer_dict, ignore_index = True)
/tmp/ipykernel_6891/864083591.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summer_dataset = summer_dataset.append(summer_dict, ignore_index = True)
/tmp/ipykernel_6891/864083591.py:52: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  summer_dataset = summer_dataset.append(summer_dict, ignore_index = True)
/tmp/ipykernel_6891/

IndexError: single positional indexer is out-of-bounds

### For the summer dataset, add the medals earned in the previous season.

In [ ]:
temp_summer_dataset = summer_dataset[["OlympicsSeason", "CountryCode", "TotalParticipants", 
                                      "GoldMedals", "SilverMedals", "BronzeMedals", "TotalMedals"]]
temp_summer_dataset = temp_summer_dataset.rename(columns = {"TotalParticipants": "PrevTotalParticipants",
                                                            "GoldMedals": "PrevGoldMedals",
                                                            "SilverMedals": "PrevSilverMedals",
                                                            "BronzeMedals": "PrevBronzeMedals",
                                                            "TotalMedals": "PrevTotalMedals"})
temp_summer_dataset["OlympicsSeason"] += 1

summer_dataset = pd.merge(summer_dataset, temp_summer_dataset, on = ["OlympicsSeason", "CountryCode"], how = "left")

summer_dataset.head()

### For summer dataset, add the winter medals (gold, silver, bronze, total) earned in the same season.

In [73]:
temp_winter_dataset = winter_dataset[["OlympicsSeason", "CountryCode", "TotalParticipants", 
                                      "GoldMedals", "SilverMedals", "BronzeMedals", "TotalMedals"]]
temp_winter_dataset = temp_winter_dataset.rename(columns = {"TotalParticipants": "PrevWinterTotalParticipants",
                                                           "GoldMedals": "PrevWinterGoldMedals",
                                                           "SilverMedals": "PrevWinterSilverMedals",
                                                           "BronzeMedals": "PrevWinterBronzeMedals",
                                                           "TotalMedals": "PrevWinterTotalMedals"})

summer_dataset = pd.merge(summer_dataset, temp_winter_dataset, on = ["OlympicsSeason", "CountryCode"], how = "left")

print(summer_dataset.shape)
summer_dataset.head()

(576, 22)


,Year,TotalParticipants,GoldMedals,SilverMedals,BronzeMedals,TotalMedals,Win%,GDPPerCapita,PopulationSize,HostCity,...,PrevWinterTotalParticipants_x,PrevWinterGoldMedals_x,PrevWinterSilverMedals_x,PrevWinterBronzeMedals_x,PrevWinterTotalMedals_x,PrevWinterTotalParticipants_y,PrevWinterGoldMedals_y,PrevWinterSilverMedals_y,PrevWinterBronzeMedals_y,PrevWinterTotalMedals_y
0,1896,5,2,0,1,3,60.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1896,8,2,1,2,5,62.5,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1896,15,1,2,3,6,40.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1896,26,5,4,2,11,42.31,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1896,25,3,3,3,9,36.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Again, check for missing values.

In [74]:
print("Summer Dataset Missing Values:", "\n\n", summer_dataset.isnull().any(), "\n")

Summer Dataset Missing Values: 

 Year                             False
TotalParticipants                False
GoldMedals                       False
SilverMedals                     False
BronzeMedals                     False
TotalMedals                      False
Win%                             False
GDPPerCapita                      True
PopulationSize                    True
HostCity                          True
OlympicsSeason                   False
CountryCode                      False
PrevWinterTotalParticipants_x     True
PrevWinterGoldMedals_x            True
PrevWinterSilverMedals_x          True
PrevWinterBronzeMedals_x          True
PrevWinterTotalMedals_x           True
PrevWinterTotalParticipants_y     True
PrevWinterGoldMedals_y            True
PrevWinterSilverMedals_y          True
PrevWinterBronzeMedals_y          True
PrevWinterTotalMedals_y           True
dtype: bool 



##### Fill in the NaN values for HostCity & all columns starting with "Prev", with 0.

In [60]:
summer_dataset.fillna(value = 0, inplace = True)

##### Fill in the NaN values for GDP and Population Size, with the median values of the entire column.

In [68]:
summer_dataset["GDPPerCapita"].replace(0, summer_dataset["GDPPerCapita"].median(), inplace = True)
summer_dataset["PopulationSize"].replace(0, summer_dataset["PopulationSize"].median(), inplace = True)

summer_dataset.head(10)

,CountryCode,Year,OlympicsSeason,TotalParticipants,GoldMedals,SilverMedals,BronzeMedals,TotalMedals,Win%,GDPPerCapita,PopulationSize,HostCity


### Import summer_dataset and winter_dataset as .csv files

In [58]:
summer_dataset.to_csv("summer_dataset.csv", index = 0)